<a href="https://colab.research.google.com/github/codefupanda/customer_interaction_summary/blob/master/Build_On_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# The basics
import numpy as np
import pandas as pd

import os
import subprocess

**Mount G-Drive to download or save files**

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


**Copy Kaggle configuration JSON file to download data from kaggle**

In [3]:
!mkdir /root/.kaggle/

In [4]:
!cp "/content/gdrive/My Drive/Colab Notebooks/.kaggle/kaggle.json" '/root/.kaggle/kaggle.json'

**Clone the code from github**

In [32]:
!git clone https://github.com/codefupanda/customer_interaction_summary.git

fatal: destination path 'customer_interaction_summary' already exists and is not an empty directory.


**Init step**: Download required data and dependencies

In [ ]:
!cd customer_interaction_summary && make requirements && make data

**TRAIN**: Train the model, src /models/model_configs.py as the configuration for which models to train

In [36]:
!cd customer_interaction_summary && git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/codefupanda/customer_interaction_summary
   e919fec..07a64f7  master     -> origin/master
Updating e919fec..07a64f7
Fast-forward
 Makefile | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [8]:
!cd customer_interaction_summary && python3 src/data/make_dataset.py data/raw data/processed

2020-10-28 06:31:12,095 - __main__ - INFO - making final data set from raw data
2020-10-28 06:31:12,128 - __main__ - INFO - Lemmatizing strings
2020-10-28 06:32:24,181 - __main__ - INFO - Splitting data
2020-10-28 06:32:24,186 - __main__ - INFO - Saving data
2020-10-28 06:32:24,217 - __main__ - INFO - successfully wrote train and test files


In [37]:
!cd customer_interaction_summary && rm -rf random_search

In [38]:
!cd customer_interaction_summary && make train

rm -rf random_search
python3 src/models/train_model.py --input_filepath=data/processed --output_filepath=models/ --pad_sequences_maxlen=1000 --max_words=30000 --epochs=20 --batch_size=128 --output_dim=300
2020-10-28 07:29:18.184864: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-28 07:29:19,435 - __main__ - INFO - starting the training process
2020-10-28 07:29:19,435 - __main__ - INFO - --input_filepath data/processed
2020-10-28 07:29:19,435 - __main__ - INFO - --output_filepath models/
2020-10-28 07:29:19,435 - __main__ - INFO - --pad_sequences_maxlen 1000
2020-10-28 07:29:19,435 - __main__ - INFO - --max_words 30000
2020-10-28 07:29:19,435 - __main__ - INFO - --epochs 20
2020-10-28 07:29:19,435 - __main__ - INFO - --batch_size 128
Found 5799 unique tokens.
Found 400000 word vectors.
Training the model: BiLSTM
2020-10-28 07:29:48.304404: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Succes

**Results are ready**

In [39]:
final_report = pd.read_csv("./customer_interaction_summary/models/final_report.csv")

In [40]:
final_report[final_report['Unnamed: 1'] == 'f1-score']

,Unnamed: 0,Unnamed: 1,1,2,3,4,5,6,7,accuracy,macro avg,weighted avg
2,BiLSTM,f1-score,0.677249,0.640145,0.461248,0.588869,0.545455,0.432432,0.393258,0.535733,0.534094,0.537299
6,BiLSTM_Glove,f1-score,0.677249,0.640145,0.461248,0.588869,0.545455,0.432432,0.393258,0.535733,0.534094,0.537299
10,StackedBiLSTM,f1-score,0.708333,0.700186,0.497529,0.621242,0.589065,0.458574,0.446125,0.574857,0.574436,0.575210
14,StackedBiLSTM_Glove,f1-score,0.708333,0.700186,0.497529,0.621242,0.589065,0.458574,0.446125,0.574857,0.574436,0.575210
18,HybridModel,f1-score,0.642599,0.696296,0.482201,0.547009,0.582734,0.429980,0.451477,0.549296,0.547471,0.551071
22,HybridModel_Glove,f1-score,0.698052,0.679054,0.466403,0.600362,0.544444,0.424125,0.440546,0.557642,0.550427,0.564800


In [41]:
final_report[final_report['Unnamed: 1'] == 'recall']

,Unnamed: 0,Unnamed: 1,1,2,3,4,5,6,7,accuracy,macro avg,weighted avg
1,BiLSTM,recall,0.655290,0.634409,0.478431,0.579505,0.554717,0.427046,0.402299,0.535733,0.533100,0.535733
5,BiLSTM_Glove,recall,0.655290,0.634409,0.478431,0.579505,0.554717,0.427046,0.402299,0.535733,0.533100,0.535733
9,StackedBiLSTM,recall,0.675497,0.714829,0.453453,0.688889,0.569966,0.485714,0.460938,0.574857,0.578469,0.574857
13,StackedBiLSTM_Glove,recall,0.675497,0.714829,0.453453,0.688889,0.569966,0.485714,0.460938,0.574857,0.578469,0.574857
17,HybridModel,recall,0.635714,0.706767,0.433140,0.514469,0.574468,0.467811,0.532338,0.549296,0.552101,0.549296
21,HybridModel_Glove,recall,0.628655,0.632075,0.508621,0.594982,0.552632,0.454167,0.470833,0.557642,0.548852,0.557642


In [42]:
final_report[final_report['Unnamed: 1'] == 'precision']

,Unnamed: 0,Unnamed: 1,1,2,3,4,5,6,7,accuracy,macro avg,weighted avg
0,BiLSTM,precision,0.700730,0.645985,0.445255,0.598540,0.536496,0.437956,0.384615,0.535733,0.535654,0.539433
4,BiLSTM_Glove,precision,0.700730,0.645985,0.445255,0.598540,0.536496,0.437956,0.384615,0.535733,0.535654,0.539433
8,StackedBiLSTM,precision,0.744526,0.686131,0.551095,0.565693,0.609489,0.434307,0.432234,0.574857,0.574782,0.579933
12,StackedBiLSTM_Glove,precision,0.744526,0.686131,0.551095,0.565693,0.609489,0.434307,0.432234,0.574857,0.574782,0.579933
16,HybridModel,precision,0.649635,0.686131,0.543796,0.583942,0.591241,0.397810,0.391941,0.549296,0.549214,0.558832
20,HybridModel_Glove,precision,0.784672,0.733577,0.430657,0.605839,0.536496,0.397810,0.413919,0.557642,0.557567,0.578038


In [43]:
!cp "./customer_interaction_summary/models/final_report.csv" "/content/gdrive/My Drive/Colab Notebooks/.kaggle/"

In [44]:
!ls "/content/gdrive/My Drive/Colab Notebooks/.kaggle/"

final_report.csv  kaggle.json  siim-isic-melanoma-classification


In [16]:
# from google.colab import files
# files.download('./customer_interaction_summary/models/final_report.csv')

In [18]:
!ls "./customer_interaction_summary/models/"

final_report.csv  StackedBiLSTM  StackedBiLSTM_hyperparameters.json


In [20]:
!cat "./customer_interaction_summary/models/StackedBiLSTM_hyperparameters.json"

{"initial_learning_rate": 0.01}